In [ ]:
import requests 
import urllib3
import os
import sys
import re
import json
from bs4 import BeautifulSoup
from lxml import html
import xml.etree.ElementTree as ET
from datetime import datetime
import openpyxl 

In [ ]:
tree = ET.parse('sitemap-produkty-telefony-i-nawigacja-gps.xml')

In [ ]:
root = tree.getroot()

In [ ]:
namespace = {'sitemap': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

In [ ]:
locs = root.findall('.//sitemap:loc', namespace)

In [ ]:
urls_list = []

In [ ]:
for loc in locs:
    if 'telefony-komorkowe' in loc.text:
        urls_list.append(loc.text)

In [ ]:
print(len(urls_list))

In [ ]:
workbook = openpyxl.Workbook()

In [ ]:
worksheet = workbook.active

In [ ]:
column_names = ['ID', 'URL', 'CALE', 'PIKSELE','WYSWIETLACZ TYP','CZESTOTLIWOSC ODSWIEZANIA EKRANU',
                'WYSWIETLACZ','APARAT','SYSTEM','PROCESOR','RAM','PAMIEC','CZYTNIK KART','MENU PL','WI-FI','ROZDZIELCZOSC WIDEO',
               'GWARANCJA','WAGA', 'BLUETOOTH','5G','BATERIA','NAZWA','CENA','WALUTA','OPINIE LICZBA', 'OCENA']
for i, name in enumerate(column_names, start=1):
    cell = worksheet.cell(row=1, column=i)
    cell.value = name


In [ ]:
count = 555
row_index = 556

for url in urls_list:
    data = []
    data.append(count)
    data.append(url)
    start_time = datetime.now()
    response = requests.get(url)
    #print(response)
    soup = BeautifulSoup(response.content, 'html.parser')
    content = response.text
    tree = html.fromstring(content)
    raw_data = tree.xpath('(//ngx-json-ld)[2]//text()')
    product_details = json.loads(*raw_data)

    value = soup.find_all('p', {'class': 'technical-attributes__attribute-value'})
    atribute = soup.find_all('p', {'class': 'technical-attributes__attribute-name'})
    values = []
    attributes = []
    for v in value:
        values.append(v.get_text())

    for a in atribute:
        attributes.append(a.get_text())
    
    list_of_attributes = ['Przekątna wyświetlacza:', 'Rozdzielczość ekranu:',
    'Rodzaj wyświetlacza:','Częstotliwość odświeżania ekranu:','Wyświetlacz:','Aparat tylny:','System operacyjny:','Procesor:',
    'Pamięć RAM:','Pamięć wbudowana:','Czytnik kart pamięci microSD:','Menu w języku polskim:','Wi-Fi:',
    'Rozdzielczość nagrywania wideo:','Gwarancja:','Waga:','Bluetooth:','5G:','Pojemność baterii:']
    m_dict = {k: v for k, v in zip(attributes, values)}
    my_dict_2 = {k: v for k, v in m_dict.items() if k in list_of_attributes}
  
    for item in list_of_attributes:
        if item in my_dict_2:
            data.append(my_dict_2[item])
        else:
            data.append('N/A')

    #nazwa
    try:
        product_name = product_details['name']
        data.append(product_name)
    except:
        data.append('N/A')

    #cena
    try:
        product_price = product_details['offers']['price']
        data.append(product_price)
    except:
        data.append('N/A')
    #waluta
    try:
        price_currency = product_details['offers']['priceCurrency']
        data.append(price_currency)
    except:
        print('N/A')
    #opinie
    try:
        review_count = product_details['aggregateRating']['reviewCount']
        data.append(review_count)
    except:
        data.append('N/A')
    try:
        avg_rating = product_details['aggregateRating']['ratingValue']
        data.append(avg_rating)
    except:
        data.append('N/A')

    data_formated = []
    for d in data:
        data_formated.append(str(d))
        
    for column_index, element in enumerate(data_formated):
        worksheet.cell(row=row_index, column=column_index+1, value=element)
        
    row_index += 1
    count+= 1
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print(f'COUNT:{count}')
    print(url)

In [ ]:
workbook.save('dane_nowe.xlsx')